# Detailed Analysis (Bid by Bid, Card by Card)

This is a tutorial of how to do a detailed analysis of a played board.

The engine looks at the bidding and play as it originally happened, and does an analysis for every bid and every card played.

The analysis is not just a double-dummy analysis for the exact current layout (like if you would press the "GIB" button on BBO). Instead, it's an analysis over many different possible layouts (samples).

In [1]:
import os
os.chdir('..')

from nn.models import Models
from analysis import CardByCard
from util import parse_lin, display_lin
from sample import Sample
import conf

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
models = Models.from_conf(conf.load('./config/default.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('./config/default.conf'))  # Load sampling strategies

INFO:tensorflow:Restoring parameters from D:\github\ben\UCBC 2024\Models\bidding-3233000
INFO:tensorflow:Restoring parameters from D:\github\ben\UCBC 2024\Models\binfo-286000
INFO:tensorflow:Restoring parameters from ..\models/lead_model_b/lead-1000000
INFO:tensorflow:Restoring parameters from ..\models/lead_model_b/lead-1000000
INFO:tensorflow:Restoring parameters from ..\models/lr3_model/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\models/lefty_model/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\models/dummy_model/dummy-920000
INFO:tensorflow:Restoring parameters from ..\models/righty_model/righty-1000000
INFO:tensorflow:Restoring parameters from ..\models/decl_model/decl-1000000


In [3]:
# we specify all the information about a board
# (it's quite tedious to enter every single thing by hand here,
# later we'll have an example of how you can give it a board played on BBO)

dealer = 'S'
vuln = [True, True]  # fist element is NS, second element is EW

hands = [
    'AJ87632.J96.753.',
    'K9.Q8542.T6.AJ74',
    'QT4.A.KJ94.KQ986',
    '5.KT73.AQ82.T532'
]

auction = ['1N', 'PASS', '4H', 'PASS', '4S', 'PASS', 'PASS', 'PASS']

play = ['C2', 'D3', 'CA', 'C6', 'D6', 'DJ', 'DQ', 'D5', 'DA', 'D7', 'DT', 'D4', 'D8', 'H6', 'H2', 'D9', 'SQ', 'S5', 'S2', 'SK', 'H4', 'HA', 'H7', 'H9', 'S4', 'C3', 'SA', 'S9', 'S3', 'C4', 'ST', 'H3', 'CK', 'C5', 'HJ', 'C7', 'C8', 'CT', 'S6', 'CJ', 'S7', 'H8', 'C9', 'D2', 'S8', 'H5', 'CQ', 'HT', 'SJ', 'HQ', 'DK', 'HK']

In [4]:
card_by_card = CardByCard(dealer, vuln, hands, auction, play, models, sampler, False)

In [5]:
# calling this starts the analysis
# it will go bid-by-bid and card-by-card, and will take a few moments
# possible mistakes will be annotated with ? or with ?? (if it's a bigger mistake)
# (possible mistake means that the engine does not agree with the bid/play. the engine could be wrong too :))

card_by_card.analyze()

analyzing the bidding
1N ? NN-value: 1C 0.997
PASS OK
4H OK
PASS OK
4S OK
PASS OK
PASS OK
PASS OK
analyzing the play
C2


Loaded lib dds.dll


D3 ?? losing: 0.81
CA OK
C6 OK
D6 OK
DJ ? losing: 0.26
DQ OK
D5 OK
DA OK
D7 OK
DT OK
D4 OK
D8 OK
H6 OK
H2 OK
D9 OK
SQ OK
S5 OK
S2 ?? losing: 0.81
SK OK
H4 OK
HA OK
H7 OK
H9 OK
S4 OK
C3 OK
SA OK
S9 OK
S3 OK
C4 OK
ST OK
H3 OK
CK OK
C5 OK
HJ OK
C7 OK
C8 OK
CT OK
S6 OK
CJ OK
S7 OK
H8 OK
C9 OK
D2 OK
S8 OK
H5 OK
CQ OK
HT OK


In [6]:
# the engine does not agree with the 1N opening.
# indeed, it's a little offbeat with a singleton
# let's see what the engine is thinking (what would it bid instead)

card_by_card.bid_responses[0].to_dict()  # the 0 index is the first bid in the auction

{'bid': '1N',
 'candidates': [{'call': '1C', 'insta_score': 0.9965}],
 'samples': ['AKJx.Qxx.Qxx.Jxx 9x.K8xxx.T8x.Txx QTx.A.KJ9x.KQ98x 8xxx.JT9x.Axx.Ax 1.00000',
  '8.Kxx.8xxx.AJTxx KJ9xx.JT8x.Axx.x QTx.A.KJ9x.KQ98x Axxx.Q9xxx.QT.xx 1.00000',
  '9xxx.K9x.Tx.AJxx AK8xx.JTxxx.A8x. QTx.A.KJ9x.KQ98x J.Q8xx.Qxxx.Txxx 1.00000',
  'K8xx.8xxx.xxx.Jx Axx.KTxx.Tx.xxxx QTx.A.KJ9x.KQ98x J9x.QJ9x.AQ8x.AT 1.00000',
  'AJ8x.Q9xxx.Ax.Tx 9xx.KJ8x.QTxx.Ax QTx.A.KJ9x.KQ98x Kxx.Txx.8xx.Jxxx 1.00000',
  'J98xx.KT8x.x.Jxx AKx.J9xxx.QTx.xx QTx.A.KJ9x.KQ98x xx.Qxx.A8xxx.ATx 1.00000',
  'A8xx.J9xx.A8x.Jx 9xx.KTxx.QTx.xxx QTx.A.KJ9x.KQ98x KJx.Q8xx.xxx.ATx 1.00000',
  'xx.Kxxxx.AT.Jxxx KJ9x.QJ9x.8xxx.x QTx.A.KJ9x.KQ98x A8xx.T8x.Qxx.ATx 1.00000',
  'Kxxx.QT.AT8.JTxx J98.J98xx.Qxx.xx QTx.A.KJ9x.KQ98x Axx.Kxxxx.xxx.Ax 1.00000',
  'AJ98x.Txx.AQxx.x Kxx.8xxx.Txx.Axx QTx.A.KJ9x.KQ98x xx.KQJ9x.8x.JTxx 1.00000',
  'KJ8x.K8xxx.Txx.J Axx.JT9x.AQx.Txx QTx.A.KJ9x.KQ98x 9xx.Qxx.8xx.Axxx 1.00000',
  'xx.QTxxx.x.ATxxx KJ8x.K8x

the engine very confidently opens `1C` and doesn't even consider `1N`

In [7]:
# what about the opening lead? let's see...

card_by_card.cards['C2'].to_dict()

{'card': 'C2',
 'candidates': [{'card': 'Cx',
   'insta_score': 0.6436,
   'expected_tricks_sd': 10.3,
   'p_make_contract': 0.75},
  {'card': 'Hx',
   'insta_score': 0.2702,
   'expected_tricks_sd': 10.31,
   'p_make_contract': 0.76},
  {'card': 'DA',
   'insta_score': 0.0617,
   'expected_tricks_sd': 10.34,
   'p_make_contract': 0.78}],
 'samples': ['QJT9xxxxx.Jx.x.9 A.Qxx.J9xx.QJxxx K8.A98x.KTxx.AK8 0.97506',
  'QJT98xxxx.xx.x.J x.J8xx.KT9xx.AQx AK.AQ9.Jxx.K98xx 0.97493',
  'KQT98xxxx.8x.9.9 J.Axx.KTxxx.QJ8x Ax.QJ9x.Jxx.AKxx 0.97210',
  'QJT9xxxx.Q98x..x A8.xx.KJT9x.QJ8x Kx.AJx.xxxx.AK9x 0.97139',
  'KQT98xxxx.xx.x.9 J.Q8x.JT9xx.AJ8x Ax.AJ9x.Kxx.KQxx 0.96968',
  'KJT8xxxx.8x.T.xx .QJ9x.J9xxx.AQ98 AQ9x.Axx.Kxx.KJx 0.96938',
  'KT98xxxx.xx.x.8x x.AQJ8.Jxxx.QJ9x AQJ.9xx.KT9x.AKx 0.96919',
  'KT98xxxx.xx.J.8x x.J8xx.K9xx.KQJ9 AQJ.AQ9.Txxx.Axx 0.96907',
  'KJTxxxxx.J.xx.8x Q.Q98x.KJTx.QJxx A98.Axxx.9xx.AK9 0.96835',
  'KQT98xxx.8x.T.xx Ax.QJ9x.J9x.QJ9x Jx.Axx.Kxxxx.AK8 0.96828',
  'KJT8x

the engine agrees with leading a low club, but it's very close. the alternative is a low heart

In [8]:
# the engine considers dummy's discard of D3 on the first trick a big mistake.
# perhaps we should ruff instead, let's see what the engine suggests

card_by_card.cards['D3'].to_dict()

{'card': 'D3',
 'candidates': [{'card': 'S3',
   'insta_score': 0.2611,
   'expected_tricks_dd': 11.19,
   'expected_score': 635},
  {'card': 'S6',
   'insta_score': 0.2611,
   'expected_tricks_dd': 11.19,
   'expected_score': 635},
  {'card': 'S2',
   'insta_score': 0.2611,
   'expected_tricks_dd': 11.19,
   'expected_score': 635},
  {'card': 'S7',
   'insta_score': 0.2611,
   'expected_tricks_dd': 11.19,
   'expected_score': 635},
  {'card': 'S8',
   'insta_score': 0.0,
   'expected_tricks_dd': 11.19,
   'expected_score': 635},
  {'card': 'SJ',
   'insta_score': 0.0,
   'expected_tricks_dd': 11.06,
   'expected_score': 631},
  {'card': 'SA',
   'insta_score': 0.0,
   'expected_tricks_dd': 10.32,
   'expected_score': 561},
  {'card': 'D5',
   'insta_score': 0.4917,
   'expected_tricks_dd': 10.38,
   'expected_score': 549},
  {'card': 'D7',
   'insta_score': 0.4917,
   'expected_tricks_dd': 10.38,
   'expected_score': 549},
  {'card': 'D3',
   'insta_score': 0.4917,
   'expected_tricks

indeed, the best play is to ruff low.

looking at the samples, we see that East has the `CA` in every sample (this is by inference because underleading an A is very unlikely)

## Analyzing a board played on BBO

In [9]:
# copy-paste from the hand records (in lin format)

lin = 'pn|You,~~M7228oka,~~M72302cm,~~M72316sq|st||md|1S4TKHJD68QC679TKA,S35H479TQKD24TAC8,S2789H3AD379JKC35,|rh||ah|Board 3|sv|e|mb|1C|an|Minor suit opening -- 3+ !C; 11-21 HCP; 12-22 total points|mb|2H|an|Aggressive weak jump overcall -- 6+ !H; 4-10 HCP |mb|d|an|Negative double -- 4+ !S; 7+ HCP; 8+ total points |mb|4H|an|The Law: 10 trump -> game support -- 4+ total points |mb|4S|an|3+ !C; 4+ !S; 16-21 HCP; 17-22 total points|mb|p|mb|p|mb|p|pg||pc|DA|pc|D3|pc|D5|pc|D6|pg||pc|C8|pc|C3|pc|CJ|pc|CA|pg||pc|S4|pc|S5|pc|S8|pc|SJ|pg||pc|H5|pc|HJ|pc|HQ|pc|HA|pg||pc|S2|pc|SA|pc|ST|pc|S3|pg||pc|H2|pc|SK|pc|H4|pc|H3|pg||pc|D8|pc|D2|pc|DJ|pc|S6|pg||pc|SQ|pc|C6|pc|H7|pc|S7|pg||pc|H8|pc|C7|pc|HK|pc|S9|pg||pc|C5|pc|C2|pc|CT|pc|HT|pg||pc|CK|pc|H9|pc|D7|pc|C4|pg||pc|DQ|pc|D4|pc|DK|pc|H6|pg||pc|D9|pc|CQ|pc|C9|pc|DT|pg||'

In [10]:
display_lin(lin)

In [11]:
board = parse_lin(lin)

In [12]:
card_by_card = CardByCard(*board, models, sampler, False)

In [13]:
card_by_card.analyze()

analyzing the bidding
1C OK
2H OK
X ? NN-value: PASS 1.000
4H OK
4S ? NN-value: 5C 0.186
PASS OK
PASS OK
PASS OK
analyzing the play
DA
D3 OK
D5 OK
D6 OK
C8 OK
C3 OK
CJ OK
CA OK
S4 OK
S5 OK
S8 OK
SJ OK
H5 OK
HJ OK
HQ OK
HA OK
S2 ? losing: 0.24
SA OK
ST OK
S3 OK
H2 OK
SK OK
H4 OK
H3 OK
D8 OK
D2 OK
DJ OK
S6 OK
SQ OK
C6 OK
H7 OK
S7 OK
H8 OK
C7 OK
HK OK
S9 OK
C5 OK
C2 OK
CT OK
HT OK
CK OK
H9 OK
D7 OK
C4 OK
DQ OK
D4 OK
DK OK
H6 OK


the engine agrees with the bidding, but didn't like something in the cardplay.

playing `S4` from hand is the first mistake. apparently this play drops almost half a trick on average.

In [14]:
card_by_card.cards['S4'].to_dict()

{'card': 'S4',
 'candidates': [{'card': 'S4',
   'insta_score': 0.0608,
   'expected_tricks_dd': 7.83,
   'expected_score': -54},
  {'card': 'DQ',
   'insta_score': 0.1129,
   'expected_tricks_dd': 7.82,
   'expected_score': -55},
  {'card': 'D8',
   'insta_score': 0.0458,
   'expected_tricks_dd': 7.82,
   'expected_score': -55},
  {'card': 'HJ',
   'insta_score': 0.3251,
   'expected_tricks_dd': 7.89,
   'expected_score': -56},
  {'card': 'ST',
   'insta_score': 0.0,
   'expected_tricks_dd': 7.78,
   'expected_score': -61},
  {'card': 'CK',
   'insta_score': 0.1072,
   'expected_tricks_dd': 7.48,
   'expected_score': -76},
  {'card': 'SK',
   'insta_score': 0.0899,
   'expected_tricks_dd': 6.98,
   'expected_score': -101},
  {'card': 'C7',
   'insta_score': 0.2223,
   'expected_tricks_dd': 6.6,
   'expected_score': -120},
  {'card': 'C6',
   'insta_score': 0.2223,
   'expected_tricks_dd': 6.6,
   'expected_score': -120},
  {'card': 'C9',
   'insta_score': 0.0211,
   'expected_tricks_d

the opening lead of `DA` is interesting. the engine prefers the `HK` and it's the only card it considers.

In [15]:
card_by_card.cards['DA'].to_dict()

{'card': 'DA',
 'candidates': [{'card': 'HK',
   'insta_score': 0.9274,
   'expected_tricks_sd': 11.13,
   'p_make_contract': 0.9}],
 'samples': ['AQT8.A.9xxx.KQ9x 9xx.J8xxx.Kx.xxx KJxx.x.QJ8.AJTxx 0.82659',
  'ATxx.A.98xx.AKTx J8x.J8xxx.xx.Jxx KQ9x.x.KQJ.Q9xxx 0.82010',
  'QJ8x.Ax.J8x.AJTx ATx.J8xxx.9x.xxx K9xx..KQxx.KQ9xx 0.81509',
  'AJTx.A.KJ98x.T9x 8xx.J8xxx.x.AJxx KQ9x.x.Qxx.KQxxx 0.80640',
  'QJ9x.Ax.QJ8.KJxx A8x.J8xxx.9xx.xx KTxx..Kxx.AQT9xx 0.79857',
  'QT8x.Ax.QJx.KQ9x A9x.J8xxx.9x.Jxx KJxx..K8xx.ATxxx 0.79848',
  'Jxxx.Ax.KQJx.KTx QTx.J8xxx.9.QJxx AK98..8xxx.A9xxx 0.78430',
  'QJxx.x.KQ8xx.K9x T9.AJ8xxx.J9.Jxx AK8xx..xx.AQTxxx 0.77924',
  'K98x.Jx.QJxx.AKT Qxx.A8xxx.9x.9xx AJTx..K8x.QJxxxx 0.77873',
  'JTxx.A.QJ8x.AJTx A9x.J8xxx.xx.xxx KQ8x.x.K9x.KQ9xx 0.77716',
  'A9xx.J8.QJ8x.AJ9 Q8x.Axxxx.xx.Txx KJTx..K9x.KQxxxx 0.77353',
  'AT8x.8.QJ9xx.AKx 9xx.AJxxx.8x.Txx KQJx.x.Kx.QJ9xxx 0.77240',
  '8xxx.x.KQJx.AJTx K9x.A8xxx.8x.9xx AQJT.J.9xx.KQxxx 0.77059',
  'AK8x..J98xx.A9xx Q9x.